In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [13]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_fl.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

/Users/ciaraspencer/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (75) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,FL,...,999 Other,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,NaN
1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,FL,...,999 Other,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14
2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,NaN
3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,FL,...,311 Food,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14
4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,FL,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,1.201109e+14
60037,265948,265948,9963928609,3/26/21,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,FL,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,1.201102e+14
60038,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,1.210103e+14
60039,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,...,999 Other,6116 Other Schools and Instruction,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,1.205700e+14


In [5]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [6]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
10719,5441437700,"42 NW 27TH AVE STE 101, MIAMI, FL",NaN,NaN
21134,6618638306,"720, GAINESVILLE, FL",NaN,NaN
57800,6905957010,"2659 E. HIGHWAY 50, CLERMONT, FL",NaN,NaN


In [7]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,9794577700,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214
1,1345247301,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510
2,1442247205,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956
3,2030068400,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300
4,2964268209,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
fips_check_scraped.head() 

/Users/ciaraspencer/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,...,999 Other,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,...,999 Other,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14
2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14
3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,...,311 Food,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14
4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [6]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9794577700,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1345247301,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14
2,1442247205,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14
3,2030068400,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14
4,2964268209,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14
...,...,...,...,...,...
60036,9950428308,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,1.201109e+14
60037,9963928609,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,1.201102e+14
60038,9977848305,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,1.210103e+14
60039,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,1.205700e+14


In [7]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
162,7392077004,"201 N. Franklin St 7th Floor, TAMPA, FL",42.706684,-73.099790,NaN
1734,9820177006,"606 Leonard C Taylor Parkway, GREEN COVE SPRIN...",41.39072,-84.132360,NaN
1933,9451747004,"200 HENDERSON RESORT WAY, DESTIN, FL",42.737133,-73.066376,NaN
3174,5003617706,"491 N State Road 7, Plantation, FL",42.698532,-73.151370,NaN
3828,4111228303,"853 State Road 436 Ste 2045, Casselberry, FL",42.70034,-73.165630,NaN
...,...,...,...,...,...
59998,9429508503,"14707 SW 42nd St Ste 406, Miami, FL",25.72962,-80.432805,NaN
60018,9645797202,"1101 TAMIAMI TRAIL STE 108, VENICE, FL",27.083321,-82.434936,NaN
60031,9870257309,"3370 HIDDEN BAY DR APT 1513, MIAMI, FL",25.951464,-80.133588,NaN
60034,9929118304,"6200 Sunset Dr Ste 304, South Miami, FL",25.703508,-80.293795,NaN


<h4>FIPS Script

In [8]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [9]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('12')] #NY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/7519 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [11]:
print(len(geo_id_list))

7519


Matching FIPS to DataFrame

In [12]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_18964/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
162,7392077004,"201 N. Franklin St 7th Floor, TAMPA, FL",42.706684,-73.099790,[]
1734,9820177006,"606 Leonard C Taylor Parkway, GREEN COVE SPRIN...",41.39072,-84.132360,[]
1933,9451747004,"200 HENDERSON RESORT WAY, DESTIN, FL",42.737133,-73.066376,[]
3174,5003617706,"491 N State Road 7, Plantation, FL",42.698532,-73.151370,[]
3828,4111228303,"853 State Road 436 Ste 2045, Casselberry, FL",42.70034,-73.165630,[]
...,...,...,...,...,...
59998,9429508503,"14707 SW 42nd St Ste 406, Miami, FL",25.72962,-80.432805,[120860152021012]
60018,9645797202,"1101 TAMIAMI TRAIL STE 108, VENICE, FL",27.083321,-82.434936,[121150024042020]
60031,9870257309,"3370 HIDDEN BAY DR APT 1513, MIAMI, FL",25.951464,-80.133588,[120860001424000]
60034,9929118304,"6200 Sunset Dr Ste 304, South Miami, FL",25.703508,-80.293795,[120860076104002]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_18964/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
162,7392077004,"201 N. Franklin St 7th Floor, TAMPA, FL",42.706684,-73.099790,NaN
1734,9820177006,"606 Leonard C Taylor Parkway, GREEN COVE SPRIN...",41.39072,-84.132360,NaN
1933,9451747004,"200 HENDERSON RESORT WAY, DESTIN, FL",42.737133,-73.066376,NaN
3174,5003617706,"491 N State Road 7, Plantation, FL",42.698532,-73.151370,NaN
3828,4111228303,"853 State Road 436 Ste 2045, Casselberry, FL",42.70034,-73.165630,NaN
...,...,...,...,...,...
59998,9429508503,"14707 SW 42nd St Ste 406, Miami, FL",25.72962,-80.432805,120860152021012
60018,9645797202,"1101 TAMIAMI TRAIL STE 108, VENICE, FL",27.083321,-82.434936,121150024042020
60031,9870257309,"3370 HIDDEN BAY DR APT 1513, MIAMI, FL",25.951464,-80.133588,120860001424000
60034,9929118304,"6200 Sunset Dr Ste 304, South Miami, FL",25.703508,-80.293795,120860076104002


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
162,7392077004,"201 N. Franklin St 7th Floor, TAMPA, FL",42.706684,-73.099790,NaN
1734,9820177006,"606 Leonard C Taylor Parkway, GREEN COVE SPRIN...",41.39072,-84.132360,NaN
1933,9451747004,"200 HENDERSON RESORT WAY, DESTIN, FL",42.737133,-73.066376,NaN
3174,5003617706,"491 N State Road 7, Plantation, FL",42.698532,-73.151370,NaN
3828,4111228303,"853 State Road 436 Ste 2045, Casselberry, FL",42.70034,-73.165630,NaN
...,...,...,...,...,...
58004,1441048808,"146 Melrose Place, Ridgewood, FL",40.979136,-74.132231,NaN
58295,7540117703,"1007 N STATE ROAD 7, ROYAL PALM BEACH, FL",42.7009,-73.170525,NaN
59513,2828308606,"7000 atrium way, Mt laurel, FL",39.915258,-74.956039,NaN
59614,3946898407,"1020 S State Road 7, Plantation, FL",42.700916,-73.170975,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
162,7392077004,NaN
1734,9820177006,NaN
1933,9451747004,NaN
3174,5003617706,NaN
3828,4111228303,NaN
...,...,...
59998,9429508503,120860152021012
60018,9645797202,121150024042020
60031,9870257309,120860001424000
60034,9929118304,120860076104002


Read in base data

In [17]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_fl.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


/Users/ciaraspencer/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,...,999 Other,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,...,999 Other,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14
2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14
3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,...,311 Food,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14
4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14


Merge with MainFrame

In [18]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,...,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN
1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,...,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14,NaN
2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,...,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14,NaN
3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,...,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14,NaN
4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,...,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,...,6211 Offices of Physicians,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,1.201109e+14,NaN
60037,60037,265948,265948,9963928609,3/26/21,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,...,7225 Restaurants and Other Eating Places,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,1.201102e+14,NaN
60038,60038,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,...,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,1.210103e+14,NaN
60039,60039,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,...,6116 Other Schools and Instruction,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,1.205700e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [19]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,...,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14,NaN,121090208023015.0
1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,...,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14,NaN,120950153001006.0
2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,...,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14,NaN,120860078082020.0
3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14,NaN,121050143012025.0
4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,...,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14,NaN,120570119082004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,...,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,1.201109e+14,NaN,120110906012023.0
60037,60037,265948,265948,9963928609,3/26/21,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,...,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,1.201102e+14,NaN,120110202041009.0
60038,60038,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,...,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,1.210103e+14,NaN,121010310082016.0
60039,60039,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,...,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,1.205700e+14,NaN,120570026001008.0


Drop Extra FIPS Columns

In [20]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,...,999 Other,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,121090208023015.0
1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",ORLANDO,...,999 Other,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,120950153001006.0
2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,120860078082020.0
3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,Lake Wales,...,311 Food,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,121050143012025.0
4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,Tampa,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,120570119082004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,120110906012023.0
60037,60037,265948,265948,9963928609,3/26/21,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,Delray Beach,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,120110202041009.0
60038,60038,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,121010310082016.0
60039,60039,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,...,999 Other,6116 Other Schools and Instruction,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,120570026001008.0


In [21]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
162,162,206073,206073,7392077004,4/7/20,455,PPP,"MORGAN & MORGAN, TAMPA, P.A.",201 N. Franklin St 7th Floor,TAMPA,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,250 to 499,"201 N. Franklin St 7th Floor, TAMPA, FL",42.706684,-73.099790,NaN
1734,1734,207645,207645,9820177006,4/9/20,491,PPP,"MABRO MARINE, INC.",606 Leonard C Taylor Parkway,GREEN COVE SPRINGS,...,999 Other,4883 Support Activities for Water Transportation,4883.0,NORTH FLORIDA DISTRICT OFFICE,c $1-2 million,100 to 249,"606 Leonard C Taylor Parkway, GREEN COVE SPRIN...",41.39072,-84.132360,NaN
1933,1933,207844,207844,9451747004,4/9/20,491,PPP,"HENDERSON BEACH RESORT HOTEL, LLC",200 HENDERSON RESORT WAY,DESTIN,...,999 Other,7211 Traveler Accommodation,7211.0,NORTH FLORIDA DISTRICT OFFICE,c $1-2 million,100 to 249,"200 HENDERSON RESORT WAY, DESTIN, FL",42.737133,-73.066376,NaN
3174,3174,209085,209085,5003617706,5/1/20,455,PPP,LEGAL AID SERVICE OF BROWARD COUNTY INC,491 N State Road 7,Plantation,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,c $1-2 million,100 to 249,"491 N State Road 7, Plantation, FL",42.698532,-73.151370,NaN
3828,3828,209739,209739,4111228303,1/22/21,491,PPS,EMPLOYU INC.,853 State Road 436 Ste 2045,Casselberry,...,999 Other,9231 Administration of Human Resource Programs,9231.0,NORTH FLORIDA DISTRICT OFFICE,c $1-2 million,250 to 499,"853 State Road 436 Ste 2045, Casselberry, FL",42.70034,-73.165630,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58004,58004,263915,263915,1441048808,4/10/21,455,PPS,DAVID GUBB,146 Melrose Place,Ridgewood,...,325 Chemicals,"3256 Soap, Cleaning Compound, and Toilet Prepa...",3256.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",Fewer than 5,"146 Melrose Place, Ridgewood, FL",40.979136,-74.132231,NaN
58295,58295,264206,264206,7540117703,5/1/20,455,PPP,MARIO THE BAKER INC,1007 N STATE ROAD 7,ROYAL PALM BEACH,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1007 N STATE ROAD 7, ROYAL PALM BEACH, FL",42.7009,-73.170525,NaN
59513,59513,265424,265424,2828308606,3/15/21,455,PPS,GOLDBIL INVESTMENT CORP,7000 atrium way,Mt laurel,...,999 Other,6244 Child Day Care Services,6244.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"7000 atrium way, Mt laurel, FL",39.915258,-74.956039,NaN
59614,59614,265525,265525,3946898407,2/5/21,455,PPS,ACCURATE PEST CONTROL INC,1020 S State Road 7,Plantation,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1020 S State Road 7, Plantation, FL",42.700916,-73.170975,NaN


Convert to File

In [22]:
fips_FL = fips_merge_drop
#fips_TN = fips_merge_drop

In [23]:
fips_FL.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/FL_fips_scraped.csv") 

Review & Compare

In [24]:
fips_FL = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/FL_fips_scraped.csv") 
fips_FL

/Users/ciaraspencer/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,6,6,9794577700,5/1/20,491,PPP,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,...,999 Other,8131 Religious Organizations,8131.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"501 State Road 13, Saint Johns, FL",30.116637,-81.625214,1.210902e+14
1,1,1,205912,205912,1345247301,4/28/20,491,PPP,"ASPIRE HEALTH PARTNERS, INC.","5151 Adanson Street, Suite 200",...,999 Other,6214 Outpatient Care Centers,6214.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"5151 Adanson Street, Suite 200, ORLANDO, FL",28.605152,-81.393510,1.209502e+14
2,2,2,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",25.68443,-80.314956,1.208601e+14
3,3,3,205914,205914,2030068400,2/3/21,455,PPP,CITRUS WORLD INC.,20205 Hwy 27,...,311 Food,3114 Fruit and Vegetable Preserving and Specia...,3114.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"20205 Hwy 27, Lake Wales, FL",27.910534,-81.602300,1.210501e+14
4,4,4,205915,205915,2964268209,8/3/20,455,PPP,"LUTHERAN SERVICES FLORIDA, INC",3627 W. WATERS AVE,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"3627 W. WATERS AVE, Tampa, FL",28.025446,-82.501656,1.205701e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60036,60036,60036,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",26.031385,-80.236000,1.201109e+14
60037,60037,60037,265948,265948,9963928609,3/26/21,455,PPP,B&B PIZZA PARTNERS LLC,125 N Congress Ave Ste 13-14,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"125 N Congress Ave Ste 13-14, Delray Beach, FL",26.253248,-80.223020,1.201102e+14
60038,60038,60038,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",28.281658,-82.675735,1.210103e+14
60039,60039,60039,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,...,999 Other,6116 Other Schools and Instruction,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",27.991997,-82.516930,1.205700e+14
